# Homework 1, Question 1

- The inspiration for this problem and the data itself is from Prof. Haugh's Quantitative Risk Management course (IEOR E4602) taught at Columbia University in 2016.
- Consider the options data in the OptionsPivotTables.xls Excel workbook.
- We want to consider joint stresses of the underlying security price and implied volatility. 
- In particular, we want to consider shifting the underlying security by -20\%, -10\%, -5\%, 0\%, +5\%, +10\% and +20\%, and shifting the implied volatility by -5, -2, 0, +2, +5, +10 and +20 volatility points. We therefore have a total of 7 × 7 = 49 scenarios.

- **Write a Python program that takes all the options data in the OptionsData worksheet, computes the P&L for each option under each scenario and outputs the results to the same
spreadsheet beginning at column AR.**

First, we count the number of assets in our portfolio. This value is saved as ``NumPosition``.

In [1]:
from openpyxl import load_workbook
from openpyxl.styles import Alignment

file_path = 'C://Users//yoshi//OneDrive//Desktop//Teaching//CFRM541 QRM//QRM_Git//QRM//H1//OptionsPivotTables.xlsm'

wb = load_workbook(filename=file_path, read_only=False, keep_vba=True)

ws = wb['OptionsData']  # or wb.active

r = 5
row = ws[r][1].value
NumPosition = 0
while row is not None:
    NumPosition += 1
    r += 1
    row = ws[r][0].value

print(NumPosition)


83


Next, we loop over each portfolio component, and insert the appropriate formula for the variation in the option price starting from cell AR5 in the OptionData worksheet.

In [2]:
# Define a lambda function to get Excel worksheet column letter from number
excel_col_name = lambda n: '' if n <= 0 else excel_col_name((n - 1) // 26) + chr((n - 1) % 26 + ord('A'))
excel_col_num = lambda a: 0 if a == '' else 1 + ord(a[-1]) - ord('A') + 26 * excel_col_num(a[:-1]) # This is to convert back to number

# Define scenarios
S = list([0.8,0.9,0.95,1,1.05,1.1,1.2])
V = list([-0.05,-0.02,0,0.02,0.05,0.1,0.2])
scenarios = [(s,v) for s in S for v in V] # list comprehension to get all scenarios as combinations of underlying and implied volatility scenarios

# Define header of scenario table and adjust column width
c = excel_col_num("AR")
for (s,v) in scenarios:
    ss = str(s)
    vv = str(v)
    ws[excel_col_name(c)+str(4)].value = f"Und {round((s-1)*100)}%, ImpVol {round(100*v)}%"
    ws.column_dimensions[excel_col_name(c)].width = 19*1.23
    ws[excel_col_name(c)+str(4)].alignment = Alignment(horizontal='center')
    c += 1


# Loop through all scenarios and fill up table with value change
for r in range(5,NumPosition):
    c = excel_col_num("AR")
    for (s,v) in scenarios:
        ss = str(s)
        vv = str(v)
        fun = f"=(BSPrice($B{r}*{s},$J{r}+{v}, $K{r}, $C{r}, ($I{r}-$B$1)/365, $G{r}, $H{r}, $D{r}) - BSPrice($B{r},$J{r}, $K{r}, $C{r}, ($I{r}-$B$1)/365, $G{r}, $H{r}, $D{r})) * $F{r}"
        ws[excel_col_name(c)+str(r)].value = fun
        ws[excel_col_name(c)+str(r)].alignment = Alignment(horizontal='center')
        ws[excel_col_name(c)+str(r)].number_format = '0'
        c += 1

# Save file                                                       
wb.save(file_path)

Finally, check the workbook

In [2]:
import os
os.system('start "excel" "OptionsPivotTables.xlsm"')

0